In [1]:
import numpy as np

import torch
import torch.optim as optim
import torch.nn as nn
from FairRanking.helpers import nDCG_cls, disparate_impact, calc_accuracy, calc_sens_loss, rND_torch, auc_estimator
from FairRanking.datasets.adult import Adult
from FairRanking.datasets.law import Law
from FairRanking.datasets.compas import Compas
from FairRanking.datasets.wiki import Wiki
from FairRanking.models.BaseDirectRanker import build_pairs1, convert_data_to_tensors
from FairRanking.models.DirectRankerAdv import DirectRankerAdv
from torch.optim.lr_scheduler import StepLR

In [2]:
#data = Law('race')
data = Adult()
#data = Compas()
#data = Wiki()
(X_train, s_train, y_train), (X_val, s_val, y_val), (X_test, s_test, y_test) = data.get_data()

In [3]:
X_train0, X_train1, s_train0, s_train1, y_train, X_val0, X_val1, s_val0, s_val1, y_val, X_test0, X_test1, s_test0, s_test1, y_test = convert_data_to_tensors(data)

/Users/robert/Desktop/Bachelor/FairRanker/FairRanking/models/BaseDirectRanker.py:205: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /private/var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_9d63z49rj_/croot/pytorch_1681837279022/work/torch/csrc/utils/tensor_new.cpp:233.)
  x0 = torch.tensor(x0, dtype=torch.float32)


In [4]:
def first_phase(model, X_train0, X_train1, y_train, optimizer, loss_fn):
    y_pred_train = model(X_train0, X_train1)
    main_loss = loss_fn(y_train, y_pred_train)
    main_loss.backward(retain_graph=True)
    optimizer.step()
    optimizer.zero_grad()
    return main_loss, y_pred_train


def second_phase(model, X_train0, X_train1, s_train0, s_train1, optimizer, loss_fn):
    sensitive_pred0, sensitive_pred1 = model.forward_2(X_train0, X_train1)
    sensitive_loss = loss_fn(torch.cat((sensitive_pred0, sensitive_pred1), dim=0), torch.cat((s_train0, s_train1), dim=0))
    #sensitive_loss = loss_fn(sensitive_pred0, s_train0)
    sensitive_loss.backward()
    #reversed_gradients = [p.grad * -1.0 for p in model.parameters() if p.grad is not None]
    for p in model.named_parameters():
        if 'debias' in p[0]:
            #print(p[1])
            continue
        else:
            if p[1].grad is not None:
                if sensitive_loss.item() < 0.41:
                    p[1].grad = torch.zeros_like(p[1].grad, dtype=torch.float32)
                    continue
                    p[1].grad *= -1
                else:
                    p[1].grad = torch.zeros_like(p[1].grad, dtype=torch.float32)

    optimizer.step()
    optimizer.zero_grad()
    return sensitive_loss

def model_rel_evaluation(model, X_val0, X_val1, s_train0, s_train1, main_loss, loss_fn, y_pred_train, y_train):
    di_train_score = disparate_impact(y_pred_train, s_train0, s_train1)
    y_val_pred = model(X_val0, X_val1)
    val_loss = loss_fn(y_val, y_val_pred)
    train_acc = calc_accuracy(y_pred_train, y_train)
    val_acc = calc_accuracy(y_val_pred, y_val)
    di_val_score = disparate_impact(y_val_pred, s_val0, s_val1)
    return main_loss.item(), train_acc, di_train_score, val_loss.item(), val_acc, di_val_score

def model_evaluation(model, X_val0, X_val1, s_val0, s_val1, y_train, y_val, main_loss, loss_fn, y_pred_train):
    y_val_pred = model(X_val0, X_val1)
    di_train_score = disparate_impact(y_pred_train, s_train0, s_train1)
    di_val_score = disparate_impact(y_val_pred, s_val0, s_val1)
    sensitive_val_pred0, sensitive_val_pred1 = model.forward_2(X_val0, X_val1)
    val_loss = loss_fn(y_val, y_val_pred)
    sensitive_val_loss = calc_sens_loss(sensitive_val_pred0, sensitive_val_pred1, s_val0, s_val1, gamma=1.0)
    train_acc = calc_accuracy(y_pred_train, y_train)
    val_acc = calc_accuracy(y_val_pred, y_val)
    return main_loss.item(), train_acc, di_train_score, val_loss.item(), val_acc, sensitive_val_loss, di_val_score

In [33]:
hidden_layers = [64, 32, 16]
#debias_layers = [32, 16, 2]
schedules = [[i,j] for j in range(1,8) for i in range(1,8)]
schedules = [[0,1]]
for schedule in schedules:
    torch.manual_seed(42)
    model = DirectRankerAdv(num_features=X_train0.shape[1],
                     kernel_initializer=nn.init.normal_,
                     hidden_layers=hidden_layers,
                     bias_layers=[128, 64, 32, 16],
             )
    #n_epochs = int(1000 / schedule[0])
    n_epochs = 1000
    lr = 0.001
    lr_decay = 0.944
    optimizer = optim.Adam(model.parameters(), lr=lr)
    adv_optimizer = optim.Adam(model.parameters(), lr=0.1)
    opt_scheduler = StepLR(optimizer, step_size=500, gamma=lr_decay)
    adv_scheduler = StepLR(adv_optimizer, step_size=500, gamma=lr_decay)
    loss_fn = nn.MSELoss(reduction='mean')
    sensitive_loss_fn = nn.BCEWithLogitsLoss()
    writer_epoch = 0 
    for epoch in range(n_epochs):
        model.train()
        for i in range(schedule[0]):
            main_loss, y_pred_train = first_phase(model, X_train0, X_train1, y_train, optimizer, loss_fn)
            model.eval()
            train_loss, train_acc, di_train_score, val_loss, val_acc, di_val_score = model_rel_evaluation(model, X_val0, X_val1, s_train0, s_train1, main_loss, loss_fn, y_pred_train, y_train)
            model.train()
            writer_epoch += 1
        for i in range(schedule[1]):
            sensitive_loss = second_phase(model, X_train0, X_train1, s_train0, s_train1, adv_optimizer, sensitive_loss_fn)
            with open('sensitive_loss.txt', 'a') as file:
                file.write(f"{writer_epoch}    {sensitive_loss.item():.4f}\n")
            model.eval()
            train_loss, train_acc, di_train_score, val_loss, val_acc, sensitive_val_loss, di_val_score = model_evaluation(model, X_val0, X_val1, s_val0, s_val1, y_train, y_val, main_loss, loss_fn, y_pred_train)
            writer_epoch += 1
            model.train()
        #opt_scheduler.step()
        #adv_scheduler.step()
        if writer_epoch % 50 == 0:
            print(f"{writer_epoch}/{n_epochs*schedule[0]+n_epochs*schedule[1]} Train Loss: {main_loss.item():.4f}  Train Accuracy: {train_acc:.4f}  Train DI: {di_train_score:.4f}\t Val Accuracy: {val_acc:.4f}  Val DI {di_val_score:.4f}")

    model.eval()
    y_test_pred = model(X_test0, X_test1)
    test_loss = loss_fn(y_test, y_test_pred)
    test_acc = calc_accuracy(y_test_pred, y_test)
    di_test_score = disparate_impact(y_test_pred, s_test0, s_test1)
    print(f'Test Loss: {test_loss.item():.4f}\t Test Accuracy: {test_acc.item():.4f}\t DI: {di_test_score:.4f}')
    print('Finished')
print("ALL FINISHED!")

50/1000 Train Loss: 0.1481  Train Accuracy: 0.9570  Train DI: 15.4622	 Val Accuracy: 0.4979  Val DI 7.8558
100/1000 Train Loss: 0.1481  Train Accuracy: 0.9570  Train DI: 15.4622	 Val Accuracy: 0.4979  Val DI 7.8558
150/1000 Train Loss: 0.1481  Train Accuracy: 0.9570  Train DI: 15.4622	 Val Accuracy: 0.4979  Val DI 7.8558
200/1000 Train Loss: 0.1481  Train Accuracy: 0.9570  Train DI: 15.4622	 Val Accuracy: 0.4979  Val DI 7.8558
250/1000 Train Loss: 0.1481  Train Accuracy: 0.9570  Train DI: 15.4622	 Val Accuracy: 0.4979  Val DI 7.8558
300/1000 Train Loss: 0.1481  Train Accuracy: 0.9570  Train DI: 15.4622	 Val Accuracy: 0.4979  Val DI 7.8558
350/1000 Train Loss: 0.1481  Train Accuracy: 0.9570  Train DI: 15.4622	 Val Accuracy: 0.4979  Val DI 7.8558
400/1000 Train Loss: 0.1481  Train Accuracy: 0.9570  Train DI: 15.4622	 Val Accuracy: 0.4979  Val DI 7.8558
450/1000 Train Loss: 0.1481  Train Accuracy: 0.9570  Train DI: 15.4622	 Val Accuracy: 0.4979  Val DI 7.8558
500/1000 Train Loss: 0.1481  

In [7]:
nDCG_cls(model, X_test0, X_test1, y_test, esti=True)

1.0

In [9]:
#import numpy as np
def calc_rnd(model, X0, X1, s0, s1):
    zero_documents = torch.zeros(size=(X0.shape[0]+X1.shape[0], X0.shape[1]))
    X_test_combined = torch.cat((X0, X1), dim=0)
    shuffled = X_test_combined[torch.randperm(X_test_combined.size(0))]
    s_test_combined = torch.cat((s0, s1), dim=0)
    model.eval()
    with torch.no_grad():
        predictions = model(X_test_combined, shuffled)
        s_test_combined = torch.argmax(s_test_combined, dim=1)
        return rND_torch(predictions, s_test_combined)

In [24]:
# rnd score in the original data
y_test_full = torch.cat((y_test, (-1)*y_test), dim=0)
s_test_full = torch.cat((s_test0, s_test1), dim=0)
base_rnd = rND_torch(y_test_full, torch.argmax(s_test_full, dim=1))

test_rnd = calc_rnd(model, X_test0, X_test1, s_test0, s_test1)
print(f"Base rND: {base_rnd}")
print(f"Test rND: {test_rnd}")

Base rND: 0.2082092918516293
Test rND: 0.08992022326871894


In [28]:
model.eval()
y_test_test = torch.zeros_like(y_test)
with torch.no_grad():
    pred = model(X_test0, X_test1)
    print(f"Accuracy: {calc_accuracy(pred, y_test)}")
    print(f"AUC: {auc_estimator(pred, y_test)}")

Accuracy: 0.5178230404853821
AUC: 0.5232743297214268


In [ ]:

print(calc_sens_loss(s_train0, s_train1, s_train0, s_train1))
bce_loss_fn = nn.BCELoss()
bce_loss_fn(torch.cat((s_train0, s_train1), dim=0), torch.cat((s_train0, s_train1), dim=0))

tensor(0.)


tensor(0.)

In [29]:
model.eval()
with torch.no_grad():
    test_bce_fn = nn.BCEWithLogitsLoss()
    sensitive_pred0, sensitive_pred1 = model.forward_2(X_val0, X_val1)
    sensitive_loss = test_bce_fn(torch.cat((sensitive_pred0, sensitive_pred1), dim=0), torch.cat((s_val0, s_val1), dim=0))
    print(sensitive_loss.item())

    

0.5588739514350891


In [39]:
torch.save(model.state_dict(), 'trained_model/testModel1.pth')

In [6]:
hidden_layers = [64, 32, 16]
debias_layers = [128, 64, 32, 16]
model = DirectRankerAdv(num_features=X_train0.shape[1],
                    kernel_initializer=nn.init.normal_,
                    hidden_layers=hidden_layers,
                    bias_layers=debias_layers)
model.load_state_dict(torch.load('trained_model/testModel1.pth'))

<All keys matched successfully>

In [35]:
from FairRanking.helpers import group_pairwise_accuracy

model.eval()
with torch.no_grad():
    pred = model(X_test0, X_test1)
    print(group_pairwise_accuracy(pred, y_test, s_test0))

0.07545982338650775
